In [1]:
import pandas as pd
import numpy as np
import os
import sqlite3
import time

In [2]:
# path to the data directory
data_path = '/mnt/DataSets/MovieLens/ml-latest-small'

# read ratings and links into dataframes
df_ratings = pd.read_csv(os.path.join(data_path,'ratings.csv'), sep=',')         
df_ratings = df_ratings.astype(float) # this, for some reason, is very important!!

print (df_ratings.head(5))
print (df_ratings.shape)

print (df_ratings.dtypes)
#ls = list(df_ratings.head(5).to_records(index=False))
#print(ls[0])
#ls[0][0].dtype

   userId  movieId  rating     timestamp
0     1.0     31.0     2.5  1.260759e+09
1     1.0   1029.0     3.0  1.260759e+09
2     1.0   1061.0     3.0  1.260759e+09
3     1.0   1129.0     2.0  1.260759e+09
4     1.0   1172.0     4.0  1.260759e+09
(100004, 4)
userId       float64
movieId      float64
rating       float64
timestamp    float64
dtype: object


In [3]:
# Create your connection.
con = sqlite3.connect(':memory:')

cur = con.cursor()
cur.execute("CREATE TABLE Ratings (UserId INTEGER, MovieId INTEGER, Rating FLOAT, timestamp DATE);")

cur.executemany("INSERT INTO Ratings (UserId, MovieId, Rating, timestamp) VALUES(?,?,?,?)", 
                list(df_ratings[['userId', 'movieId', 'rating', 'timestamp']].to_records(index=False)))

query ="SELECT * from Ratings Limit 5"
cur.execute(query)
rows= cur.fetchall()
for row in rows:
    print (row)

df_out = pd.read_sql_query("SELECT * from Ratings Limit 5",con)
df_out.dtypes

(1, 31, 2.5, 1260759144)
(1, 1029, 3.0, 1260759179)
(1, 1061, 3.0, 1260759182)
(1, 1129, 2.0, 1260759185)
(1, 1172, 4.0, 1260759205)


UserId         int64
MovieId        int64
Rating       float64
timestamp      int64
dtype: object

In [4]:
start_time = time.time()

# 1. compute item-item co-occurrences thresholded
cur.execute("CREATE TABLE cooc AS select a.MovieId as ItemId1, b.MovieId as ItemId2, count(*) as cooc \
            from ratings as a, ratings as b \
            where a.UserId = b.UserId and a.MovieId > b.MovieId \
            group by ItemId1, ItemId2 \
            having cooc >= 3")

# 2. compute item occurrences
cur.execute("CREATE TABLE item_counts AS select MovieId, count(*) as occur from ratings group by MovieId order by occur desc;")

#3. compute item-item jaccard similarity
cur.execute("CREATE TABLE jaccard AS select t1.ItemId1, t1.ItemId2, t1.cooc, \
           t2.occur as occur1, t3.occur as occur2, \
           (1.0*t1.cooc/(t2.occur+t3.occur-t1.cooc)) as jaccard \
            from item_counts t2 inner join cooc t1 on t1.ItemId1 = t2.MovieId inner join item_counts t3 on t1.ItemId2 = t3.MovieId \
            order by t1.ItemId1, jaccard desc;")

#4. convert item similarities to df
df_jaccard = pd.read_sql_query("SELECT * from jaccard", con)
df_mpi = pd.read_sql_query("SELECT MovieId as ItemId, occur \
                                    from item_counts \
                                    order by occur desc \
                                    limit 10", con)
#5. clean up teh sql tables and close the connection
cur.execute("DROP TABLE cooc")
cur.execute("DROP TABLE item_counts")
cur.execute("DROP TABLE jaccard")

#con.commit()
con.close()

print("--- It took %s seconds ---" % (time.time() - start_time))

--- It took 31.319721937179565 seconds ---


cur.execute("SELECT * FROM jaccard LIMIT 5")
rows= cur.fetchall()
for row in rows:
    print (row)

In [5]:
print("----------Item-Item Similarties---------------")
print(df_jaccard.shape)
print(df_jaccard.head(5))
print(df_jaccard.dtypes)
num_items = pd.concat([df_jaccard.ItemId1, df_jaccard.ItemId2],axis = 0).unique().shape[0]
print("Number of Unique Items:",num_items)
print("----------Most Popular Items---------------")
print(df_mpi.head(5))

----------Item-Item Similarties---------------
(2208878, 6)
   ItemId1  ItemId2  cooc  occur1  occur2   jaccard
0        2        1    71     107     247  0.250883
1        3        1    39      59     247  0.146067
2        3        2    18      59     107  0.121622
3        4        3     5      13      59  0.074627
4        4        2     5      13     107  0.043478
ItemId1      int64
ItemId2      int64
cooc         int64
occur1       int64
occur2       int64
jaccard    float64
dtype: object
Number of Unique Items: 4792
----------Most Popular Items---------------
   ItemId  occur
0     356    341
1     296    324
2     318    311
3     593    304
4     260    291


In [6]:
from SAR_scoring import SAR_Score

start_time = time.time()

# read in the seed file
df_seeds = pd.read_csv(os.path.join(data_path,'SecondToLastItemPerUser.csv'), sep=',', 
                       header = None, names=['UserId','ItemId','Aff'])

df_scores = SAR_Score(simDF = df_jaccard, usageDF = df_seeds)   
print(df_scores.head(5))
print("\nFreq of Flag:\n",df_scores.Flag.value_counts())
print("\n--- It took %s seconds ---" % (time.time() - start_time))

DEBUG: There are 4792 unique items.

DEBUG: Most Popular Items:
DEBUG: 356	341
DEBUG: 296	324
DEBUG: 318	311
DEBUG: 593	304
DEBUG: 260	291

DEBUG: Sparsity of Item Similarity Matrix%:  19.238362629981523

DEBUG: There are 671 users to score.

  UserId ItemId     Score Flag
0      1   2105  0.802469    R
1      1   3108  0.573477    R
2      1   2872  0.558239    R
3      1   1129  0.556425    R
4      1   2021  0.554473    R

Freq of Flag:
 R    3021
T     334
Name: Flag, dtype: int64

--- It took 11.551105260848999 seconds ---


In [7]:
# convert the <UserId, ItemId, prediction> format to horizontal: <UserId, Reco1, Reco2, ..., Reco5>
k_max = 5
D_recos = []
pre = ""
sim_items = []
k = 1
for row in df_scores.itertuples():
    UserId = str(row[1])
    ItemId = str(row[2])
    
    # for a new user block" start except the very first one
    if (pre == ""):
        sim_items.append(UserId)
    if (pre != UserId and pre != ""): 
        D_recos.append(sim_items)
        sim_items = []
        sim_items.append(UserId) 
        k = 1
    if(k <= k_max):
        pre=UserId   
        sim_items.append(ItemId)
        k +=1
    else:
        pass
# last item related items
D_recos.append(sim_items)

df_recos_h = pd.DataFrame(D_recos, dtype='str')
df_recos_h.to_csv(data_path + '/pred_horizontal_SAR.csv', index=False, header=False)

In [8]:
%%bash
cd /mnt/DataSets/MovieLens/ml-latest-small

# Compute Precision@1-5 on out of time sample Test file with an external python script
for k in 1 2 3 4 5; do
    /home/vediev/bin/recPrecision_Arg.py -t lastItemPerUser.csv -p pred_horizontal_SAR.csv -k ${k}
done

cat pred_horizontal_SAR.csv | cut -d, -f2- | tr "," "\n" | sort | uniq | wc -l

k=1
precision_avg = 0.022355
counted user number = 671
k=2
precision_avg = 0.032787
counted user number = 671
k=3
precision_avg = 0.041729
counted user number = 671
k=4
precision_avg = 0.053651
counted user number = 671
k=5
precision_avg = 0.061103
counted user number = 671
1210
